In [1]:
import numpy as np


In [2]:
# step 1: load data

iris_data = np.loadtxt('../dataset/iris.txt')

In [3]:
# step 2: split the data in training data and test data

from utility import split_data
x_train, y_train, x_test, y_test = split_data(iris_data[:, :4], iris_data[:, 4])

In [4]:
# step 3-1: calculate within-class scatter and between-class scatter of training set and calculate Fisher's score

class Performance_index():
    def __init__(self, data):
        self.data = data

    def get_fisher_score(self):
        within_class_scatter = self.get_between_class_scatter()
        between_class_scatter = self.get_between_class_scatter()

        performance_index = self.get_between_class_scatter() / self.get_within_class_scatter()
        fisher_score = [performance_index[index, index] for index in range(performance_index.shape[0])]
        
        return fisher_score
    
    def get_within_class_scatter(self):
        within_class_scatter = 0
        for class_data in [self.data[:, :25], self.data[:, 25:50], self.data[:, 50:75]]:
            prior_probability = class_data.shape[1] / self.data.shape[1]
            data_count = class_data.shape[1]
            mean_vector = np.mean(class_data, axis=1)
            # print(mean_vector.shape)

            data_scatter = 0
            for data_index in range(data_count):
                data_minus_mean_vector = np.expand_dims((class_data[:, data_index] - mean_vector), axis=1)
                data_scatter += data_minus_mean_vector.dot(data_minus_mean_vector.T)
            
            within_class_scatter += data_scatter * prior_probability / data_count

        return within_class_scatter

    def get_between_class_scatter(self):
        between_class_scatter = 0
        mean_vector = np.expand_dims(self.data.mean(axis=1), axis=1)
        for class_data in [self.data[:, :25], self.data[:, 25:50], self.data[:, 50:75]]:
            prior_probability = class_data.shape[1] / self.data.shape[1]
            class_mean_vector = np.expand_dims(class_data.mean(axis=1), axis=1)
            between_class_scatter += (class_mean_vector - mean_vector).dot((class_mean_vector - mean_vector).T)
        
        return  prior_probability * between_class_scatter

performance_index = Performance_index(x_train.T)
fisher_score = performance_index.get_fisher_score()

for feature_index in range(len(fisher_score)):
    print("The feature {}'s fisher score is: {}".format(feature_index + 1, fisher_score[i]))

NameError: name 'i' is not defined

In [5]:
# step 4 ~ 7 take top-X features, calc average accuracy by using K-NN model with two-fold cross validation

from utility import get_model_accuracy

feature_rank = [sorted(fisher_score).index(x) for x in fisher_score][::-1]
for rank_index in range(len(feature_rank)):
    rank_index += 1
    selected_features = feature_rank[:rank_index]
    accuracy = get_model_accuracy(x_train[:, selected_features], y_train, x_test[:, selected_features], y_test)
    print("When use top-{} features, accuracy is: {}".format(rank_index, accuracy))

When use top-1 features, accuracy is: 0.9199999999999999
When use top-2 features, accuracy is: 0.9533333333333334
When use top-3 features, accuracy is: 0.9533333333333334
When use top-4 features, accuracy is: 0.94
